# Code for CLa&ssification of toric colorable seeds of Picard number $4$
In terms of:
    - Positivity
    - Fan-givingness
    - Projectivity

In [2]:
import SimplicialComplex as sc
import json
import sympy as sp

def read_file(filename):
    with open(filename, 'rb') as f:
        data = f.readlines()
        data = [x.strip() for x in data]
    return data

### Function for loading all the seeds of a given pair $(m,n)$

In [3]:
def load_seeds(n,m):
    db_path = 'final_results/CSPLS_%d_%d' % (n, m)
    list_facets = [json.loads(facets_bytes) for facets_bytes in read_file(db_path)]
    return [sc.PureSimplicialComplex(facets_set for facets_set in list_facets)]

    
    

### Function for testing positivity of a pair $(K,\lambda^{\mathbb{R}})$

In [9]:
def is_positive(K:sc.PureSimplicialComplex,mod_2_IDCM):
    (m,n) = (K.m,K.n)
    poly_char_map = sp.Matrix((m-n,m),dtype=object)
    x_vars = sp.symarray('x',(m-n,n))
    for i in range(m-n):
        for j in range(m):
            poly_char_map[i,j] = mod_2_IDCM[i,j]
    poly_char_map[:,n:] += 2*x_vars
    list_cofacets = []
    for facet_bin in K.facets_bin:
        cofacet_bin = facet_bin ^ (sc.list_2_pow[m]-1)
        list_cofacets.append(sc.binary_to_face(cofacet_bin,m))
    N = len(list_cofacets)

    system = []
    for i in range(N):
        for j in range(i+1,N):
            intersection = sc.binary_to_face(K.facets_bin[i]&K.facets_bin[j])
            if len(intersection)==n-1:
                vertcices = sc.binary_to_face(K.facets_bin[i]^K.facets_bin[j])
                intersection1 = intersection.copy()
                intersection1.append(vertcices[0])
                intersection2 = intersection.copy()
                intersection2.append(vertcices[1])
                system.append(poly_char_map[:,intersection1]*poly_char_map[:,intersection2])
    min_ones = -sp.ones(len(system))
    sp.solve(system,min_ones)
    

    
